# Bank Churn Exercise - Building ANN

* Bank has been facing churn issue, measured the results 6 months ago
* Did the person leave or stay?
* Goal: ANN model, to predict which customers are highly likely to leave versus staying

In [57]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [58]:
# Check TF version:
tf.__version__

'2.17.0'

In [59]:
# Import dataset
dataset = pd.read_csv('Churn_Modelling.csv')

In [60]:
# Let's look at the dataset:
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:, -1].values

In [61]:
# Notes:
# Import everything, include TF package
# The Customer ID is the key identifier, has no impact on dependent variable

# Does Surname have impact on Churn? Likely no

# Everything else will be kept, we will remove the first 2 

In [62]:
print(y)

[1 0 1 ... 1 1 0]


In [63]:
# Is there any missing data?
dataset.info()

# No missing data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [64]:
# Will need to encode some variables
from sklearn.preprocessing import LabelEncoder

In [65]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [66]:
y

array([1, 0, 1, ..., 1, 1, 0])

In [67]:
# Call Encoder
le = LabelEncoder()

# We want to encode
# This is the 2nd column from the new truncated dataset

# Label Encoder will fail if you don't pass values into it, not the whole DF.
X[:,2] = le.fit_transform(X[:,2])

In [79]:
df = dataset.iloc[:,3:-1].copy()

In [80]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [71]:
# One Hot Encoding and transformers:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [82]:
# One Hot encode - geography columns:
ct = ColumnTransformer(transformers= [('encoder', OneHotEncoder(), [1])], remainder = "passthrough")
# 1 = Geography is in column position 1 (counting from zero)

# passthrough means to apply transformation but do not drop the original columns
# Use this method instead of pandas - pd.encode()

In [83]:
# Apply the ct, column transformer we just made:
X = np.array(ct.fit_transform(X))

In [84]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [86]:
# Split X-train and X-test:
from sklearn.model_selection import train_test_split

# 20% testing, 42 random state
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [87]:
# Feature Scaling is mandatory, as deep learning is sensitive to larger ranges and outliers.
# Similar to Normal distribution
from sklearn.preprocessing import StandardScaler

In [88]:
# Apply the scale:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

# Part 2 - Build the Artificial Neural Network

In [89]:
# Set up the ANN
# Sequential layer - ANN is a sequence of layers, other type is computational graph

# TensorFlow and Keras are combined together.

# Make an instance, object, sequential
ann = tf.keras.models.Sequential()

In [90]:
# Add the input layer, and add the hidden layer
# Use the Dense class

# How to add fully connected:
ann.add(tf.keras.layers.Dense(units = 6, activation = "relu"))

# Several arguments:
# units = number of neurons in want in that specific layer.
# First Hidden Layer = 6 neurons

# do not confuse independent variables 
# How do you know how many hidden layers to have - no, you need to experiment with hyperparameters
# The instructor tried several numbers, it ended up being 6.
# Activation = Rectifier Function is the most popular one.

In [92]:
# Add the second hidden layer
ann.add(tf.keras.layers.Dense(units = 6, activation = "relu"))

In [93]:
# Add the output layer
# Units = 1, since we are doing binary categorical variable.
# If it's categorical, it'd be unit = 3

# This depends on the nature of your final
ann.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# soft max -> if not binary, like categorical var predictions, change activation to soft max

# Why Sigmoid? It gets you the prediction, and the prediction that the binary outcome is 1.
# You get the probability number.

# Part 3: Train the ANN

In [95]:
# Compile the ANN
ann.compile(optimizer= "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

# Parameters;
# Metrics - classification, let's usee accuracy. You can pick accuracy, precision and recall, or f1-score.
# optimizer - best to use is SDG - "adam", it does SDG, which is the best for ANN
# Loss function (Cost) - when doing binary classification, the loss function must be binary_crossentropy

# If not binary, like categorical variable => category_crossentropy

In [96]:
# Train the ANN on training set
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

# Parameters:
# batch_size = batch learning is more better for performance efficiency
# epochs = certain number of epochs, we will use 100

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 289us/step - accuracy: 0.6904 - loss: 0.6296
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 254us/step - accuracy: 0.7927 - loss: 0.4782
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 248us/step - accuracy: 0.7944 - loss: 0.4514
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 244us/step - accuracy: 0.8118 - loss: 0.4392
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 230us/step - accuracy: 0.8202 - loss: 0.4312
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 227us/step - accuracy: 0.8158 - loss: 0.4291
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 442us/step - accuracy: 0.8187 - loss: 0.4236
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 887us/step - accuracy: 0.8225 - loss: 0.4211
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 384us/step - accuracy: 0.8251 - loss: 0.4100
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 270us/step - accuracy: 0.8241 - loss: 0.4118
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 292us/step - accuracy: 0.8179 - loss: 0.4265
Epoch 12/100
250/25

In [103]:
# Object is ann
X_test[0]

array([-1.02020406,  1.73668197, -0.56349184, -0.51250172,  0.90911166,
       -0.67729931, -0.69844549,  0.30084641,  0.80510537, -1.57810057,
       -1.0502616 , -0.99096946])

# Part 4 - Use ANN for prediction

In [ ]:
# Task:
# French Customer
# Credit Score = 600
# Gender = Male
# Age: 40 years old
# Tenure: 3 years
# Balance: 60,000
# Number of Products: 2
# Credit Card = Yes
# Active member = yes
# Estimated Salary = 50,000

# Is this customer expected to leave our bank? Churn Y/N?

In [101]:
# Prediction set:
prediction_set = np.array([1])